In [1]:
import pandas as pd
import re
import unidecode

In [2]:
# 1) Upload CSV file containing scraped raw data (web_scraping.ipynb).

# Specifies directory path where CSV file is located.
directory = 'C:/Users/DAV/Documents/Python/Python_Project/House_price_project/data/raw/'

# Define file name.
filename = 'raw-data.csv'  

# Build full file path.
file_path = directory + filename

# Upload the CSV file using constructed path.
df = pd.read_csv(file_path)

In [3]:
# 2) Replace NaN with 0. 
df['square_meters'] = df['square_meters'].fillna(0)
df['rooms'] = df['rooms'].fillna(0)
df['parking'] = df['parking'].fillna(0)
df['baths'] = df['baths'].fillna(0)

In [4]:
# 3) Convert columns to integers.
df['square_meters'] = df['square_meters'].astype(int)
df['rooms'] = df['rooms'].astype(int)
df['parking'] = df['parking'].astype(int)
df['baths'] = df['baths'].astype(int)

In [5]:
# 4) Delete those records that have similarity in: title, square_meters, rooms, parking and baths.

#  Count rows number before removing duplicates.
initial_count = df.shape[0]

# Remove duplicates considering specified columns.
df.drop_duplicates(subset=['title', 'square_meters', 'rooms', 'parking', 'baths'], inplace=True)

# Reset index.
df.reset_index(drop=True, inplace=True)

# Count rows number after removing duplicates.
final_count = df.shape[0]

# Calculate how many rows were deleted.
removed_count = initial_count - final_count

# Message about how many rows were deleted.
print(f"\n{removed_count} duplicate rows were removed.")


27 duplicate rows were removed.


In [6]:
# 5) Delete rows containing less than 40 square meters.

#  Count rows number before removing duplicates.
initial_count = df.shape[0]
# Filter rows.
df = df[df['square_meters'] >= 40]
# Reset index.
df.reset_index(drop=True, inplace=True)
# Count rows number after removing duplicates.
final_count = df.shape[0]
# Calculate how many rows were deleted.
removed_count = initial_count - final_count
# Message about how many rows were deleted.
print(f"\n{removed_count} rows were removed.")


58 rows were removed.


In [7]:
# 6) Convert price to integer, removing symbols ("[ ]",","$","Q") and applying a conversion rate.

# Define conversion rate ($ to Q).
conversion_rate = 7.8

def convert_price(price):
    # Remove extra spaces and brackets, and take first value in the list.
    price = price.strip("[]'")  # Remove brackets and quotes.

    # Check if price is in quetzales or dollars.
    if 'Q' in price:
        return int(price.replace('Q', '').replace(',', ''))  # Convert to integer.
    elif '$' in price:
        # Convert price from dollars to quetzales adjustable based on current exchange rate).
        usd_value = int(price.replace('$', '').replace(',', ''))  # Convert to integer.
        return int(usd_value * conversion_rate)  
    else:
        return None  # Or handle another way if it is not in the expected format

# Apply function and create a new column 'price_in_quetzales'
df['price_quetzales'] = df['price'].apply(convert_price)

In [8]:
# 7) Normalize text function.
def normalize_text(text):
    if isinstance(text, str):
        text = re.sub(r"[.,|()*!-/•:?]", " ", text)  # Replace ".", ",", "|", etc. for space.
        text = re.sub(r'\b(zona)\s*(\d+)', r'\1 \2', text)  # Ensure space between "zone" and number.
        text = re.sub(r'(\d+)([a-zA-Z])', r'\1 \2', text)  # Space between number and letter.
        text = re.sub(r'\s+', ' ', text).strip()  # Delete multiple spaces.
        text = text.lower().replace("blvd", "boulevard")  # Replace "blvd" to "boulevard"
        return unidecode.unidecode(text.strip())
    return ''  # Return empty string if not a string.

# Normalize titles.
df['title_normalized'] = df['title'].apply(normalize_text)
# Normalize description.
df['description_normalized'] = df['adv_description'].apply(normalize_text)
# Normalize location.
df['location_normalized'] = df['location'].apply(normalize_text)

In [9]:
# 8) Define a new column order.
new_order = ['title_normalized', 'price_quetzales','square_meters', 'rooms', 'parking', 'baths', 'location_normalized', 'description_normalized']

# Rearrange DataFrame according to the new order.
df = df[new_order]

In [10]:
# 9) Save Clean Data.
# Specify directory path.
directory = 'C:\\Users\\DAV\\Documents\\Python\\Python_Project\\House_price_project\\data\\raw\\'

# Define the file name.
filename = 'pre-clean.csv'

# Build full file path.
output_path = directory + filename

# Save DataFrame to an CSV file in the specified directory.
df.to_csv(output_path, index=False)